In [5]:
import pandas as pd
from pathlib import Path

# Put your CSV files here
csv_paths = [
    "C:\\Users\\wongb\\user_trajectories_winter\\494-user-trajectories\\students\\benedict\\data\\labled_cn\\cn_sample_1.csv",
    "C:\\Users\\wongb\\user_trajectories_winter\\494-user-trajectories\\students\\benedict\\data\\labled_cn\\cn_sample_3.csv"
]


# Where to save the combined CSV
output_path = "C:\\Users\\wongb\\user_trajectories_winter\\494-user-trajectories\\students\\benedict\\data\\labled_cn\\combined.csv"

combined_df = pd.concat([pd.read_csv(path) for path in csv_paths], ignore_index=True)
Path(output_path).parent.mkdir(parents=True, exist_ok=True)
combined_df.to_csv(output_path, index=False)

print(f"Saved {len(combined_df)} rows to {output_path}")

Saved 80 rows to C:\Users\wongb\user_trajectories_winter\494-user-trajectories\students\benedict\data\labled_cn\combined.csv


In [ ]:
import pandas as pd
import openai
import os

# Function to classify tweets based on partisan lean
def classify_tweets(csv_path):
    # Load the CSV file
    df = pd.read_csv(csv_path)
    
    # Check if 'full_text' column exists
    if 'full_text' not in df.columns:
        raise ValueError("The CSV must contain a 'full_text' column.")
    
    # Initialize OpenAI client with API key from environment variable
    api_key = os.getenv('OPENAI_API_KEY')
    if not api_key:
        raise ValueError("OPENAI_API_KEY is not set in the environment.")
    client = openai.OpenAI(api_key=api_key)
    
    # Function to get partisan lean from OpenAI
    def get_partisan_lean(tweet):
        prompt = f"""Classify the following tweet as LEFT, RIGHT, or APOLITICAL based on the ideological disposition of the speaker/writer.

DEFINITIONS:
- L: Any position from liberal democratic to communist, and anything in between. Includes liberal democrat, progressive, socialist, social democratic, anti-colonialist and anti-imperialist, anarchist, Marxist, internationalist, and communist ideologies.
- R: Any position from liberal conservative to fascist, and anything in between. Includes conservative, libertarian-right, authoritarian-right, monarchist, nationalist, anti-woke, religious conservatism, anti-globalism, racist, sexist, and fascist ideologies.
- A: Unopinionated, objective statements such as news reporting, can be false info with no opinion, scientific statements, factual observations, commonly held apolitical opinions or statements, or purely informational or advertising content with no ideological framing. However, tone and reason for pointing certain events out must be checked to see if the writer views this news as good or not potentially indicating a partisan leaning.

IMPORTANT GUIDANCE:
Judge based on the ideological disposition of the speaker/writer, not based on a checklist of specific positions. Take into account internet style subcultures, memes, coded language, and subcultural references that may convey ideological meaning beyond surface-level keywords. Consider tone, framing, and the broader context of online discourse. Answer the question based on who would write this?

Tweet to classify: {tweet}
Return ONLY ONE of the following words as your answer: L, R, A. Do not provide any additional explanation or text.
"""
        response = client.chat.completions.create(
            model='gpt-4o-mini',
            messages=[{'role': 'user', 'content': prompt}]
        )
        return response.choices[0].message.content
    
    # Apply the classification to each tweet
    df['llm_label'] = df['full_text'].apply(get_partisan_lean)
    
    return df


df = classify_tweets("C:\\Users\\wongb\\user_trajectories_winter\\494-user-trajectories\\students\\benedict\\data\\labled_cn\\combined.csv")
df


,post_id,full_text,summary,rating,tweetId,noteId,llm_label
0,1.763800e+18,Ramanujan: It came to me in my dream.\r\n\r\nT...,The formulas (1) and (4) (which are actually t...,A,NaN,NaN,A
1,1.939840e+18,what. what. what do you mean. https://t.co/mis...,"The above post is a joke, showing a fabricated...",A,NaN,NaN,A
2,1.751640e+18,@RealAlexJones From The Beginning InfoWars Rep...,"No vaccine, including the Covid vaccines, caus...",R,NaN,NaN,R
3,1.777570e+18,This one community noter on a power trip is ca...,Community notes are not captured by one person...,R,NaN,NaN,A
4,1.698360e+18,I’ve said since day 1 that I believe we revisi...,The quoted image is from an Aug 2017 blog http...,R,NaN,NaN,A
...,...,...,...,...,...,...,...
75,NaN,JUST IN - Hunter Biden tax charges dismissed b...,NNN. Post is correct and context is found in t...,A,1.692310e+18,1.692370e+18,A
76,NaN,I didn’t expect to get asked this on my Saturd...,According to various articles including one li...,R,1.388590e+18,1.388700e+18,R
77,NaN,🚨FAKE NEWS from NBC: White House Officials con...,A White House official said in a text message ...,R,1.882960e+18,1.883110e+18,R
78,NaN,❌ We are witnessing a radical change in the ci...,Analysts say Russia's move means it will becom...,R,1.699470e+18,1.699500e+18,L


In [7]:
df.to_csv("C:\\Users\\wongb\\user_trajectories_winter\\494-user-trajectories\\students\\benedict\\data\\labled_cn\\combined_llm.csv", index=False)

In [9]:
import pandas as pd

# Simple comparison: ground truth vs LLM label
comparison = pd.DataFrame({
    'rating': df['rating'],
    'llm_label': df['llm_label']
})

comparison['match'] = comparison['rating'] == comparison['llm_label']

# Overall accuracy summary
summary = pd.DataFrame({
    'Metric': ['Matches', 'Total', 'Accuracy'],
    'Value': [
        int(comparison['match'].sum()),
        len(comparison),
        f"{comparison['match'].mean():.2%}"
    ]
})

# Accuracy per label type (based on ground-truth rating)
per_type_accuracy = (
    comparison.groupby('rating', dropna=False)['match']
    .mean()
    .reset_index(name='accuracy')
    .sort_values('rating')
)
per_type_accuracy['accuracy'] = per_type_accuracy['accuracy'].map(lambda x: f"{x:.2%}")

summary, per_type_accuracy

(     Metric   Value
 0   Matches      51
 1     Total      80
 2  Accuracy  63.75%,
   rating accuracy
 0      A   76.19%
 1      L   57.14%
 2      R   60.00%)

In [ ]:
import pandas as pd
import openai
import os

# Function to classify inferred tweet political leaning from community notes
def classify_tweets(csv_path):
    # Load the CSV file
    df = pd.read_csv(csv_path)
    
    # Check if 'summary' column exists
    if 'summary' not in df.columns:
        raise ValueError("The CSV must contain a 'summary' column.")
    
    # Initialize OpenAI client with API key from environment variable
    api_key = os.getenv('OPENAI_API_KEY')
    if not api_key:
        raise ValueError("OPENAI_API_KEY is not set in the environment.")
    client = openai.OpenAI(api_key=api_key)
    
    # Function to infer tweet political leaning from a community note summary
    def get_partisan_lean(note_summary):
        prompt = f"""You are given a Community Note summary written in response to a tweet. Infer the likely political leaning of the ORIGINAL tweet author based on this note.

Classify as LEFT, RIGHT, or APOLITICAL based on the ideological disposition of the original tweet's speaker/writer.

DEFINITIONS:
- L: Any position from liberal democratic to communist, and anything in between. Includes liberal democrat, progressive, socialist, social democratic, anti-colonialist and anti-imperialist, anarchist, Marxist, internationalist, and communist ideologies.
- R: Any position from liberal conservative to fascist, and anything in between. Includes conservative, libertarian-right, authoritarian-right, monarchist, nationalist, anti-woke, religious conservatism, anti-globalism, racist, sexist, and fascist ideologies.
- A: Unopinionated, objective statements such as news reporting, can be false info with no opinion, scientific statements, factual observations, commonly held apolitical opinions or statements, or purely informational or advertising content with no ideological framing. However, tone and reason for pointing certain events out must be checked to see if the writer views this news as good or not potentially indicating a partisan leaning.

IMPORTANT GUIDANCE:
You are NOT classifying the Community Note author. You are inferring the likely ideological disposition of the original tweet author from the note's content, framing, and what claim is being corrected or contextualized. Take into account internet style subcultures, memes, coded language, and subcultural references that may convey ideological meaning beyond surface-level keywords. If evidence is weak or ambiguous, choose APOLITICAL.

Community Note summary: {note_summary}

Return ONLY ONE of the following words as your answer: L, R, A. Do not provide any additional explanation or text.
"""
        response = client.chat.completions.create(
            model='gpt-4o-mini',
            messages=[{'role': 'user', 'content': prompt}]
        )
        return response.choices[0].message.content
    
    # Apply the classification to each note summary
    df['llm_label_via_note'] = df['summary'].apply(get_partisan_lean)
    
    return df


df_note = classify_tweets("C:\\Users\\wongb\\user_trajectories_winter\\494-user-trajectories\\students\\benedict\\data\\labled_cn\\combined_llm.csv")
df_note


,post_id,full_text,summary,rating,tweetId,noteId,llm_label,llm_label_via_note
0,1.763800e+18,Ramanujan: It came to me in my dream.\r\n\r\nT...,The formulas (1) and (4) (which are actually t...,A,NaN,NaN,A,A
1,1.939840e+18,what. what. what do you mean. https://t.co/mis...,"The above post is a joke, showing a fabricated...",A,NaN,NaN,A,A
2,1.751640e+18,@RealAlexJones From The Beginning InfoWars Rep...,"No vaccine, including the Covid vaccines, caus...",R,NaN,NaN,R,A
3,1.777570e+18,This one community noter on a power trip is ca...,Community notes are not captured by one person...,R,NaN,NaN,A,A
4,1.698360e+18,I’ve said since day 1 that I believe we revisi...,The quoted image is from an Aug 2017 blog http...,R,NaN,NaN,A,A
...,...,...,...,...,...,...,...,...
75,NaN,JUST IN - Hunter Biden tax charges dismissed b...,NNN. Post is correct and context is found in t...,A,1.692310e+18,1.692370e+18,A,A
76,NaN,I didn’t expect to get asked this on my Saturd...,According to various articles including one li...,R,1.388590e+18,1.388700e+18,R,L
77,NaN,🚨FAKE NEWS from NBC: White House Officials con...,A White House official said in a text message ...,R,1.882960e+18,1.883110e+18,R,A
78,NaN,❌ We are witnessing a radical change in the ci...,Analysts say Russia's move means it will becom...,R,1.699470e+18,1.699500e+18,L,A


In [12]:
import pandas as pd

# Simple comparison: ground truth vs LLM label
comparison = pd.DataFrame({
    'rating': df_note['rating'],
    'llm_label': df_note['llm_label_via_note']
})

comparison['match'] = comparison['rating'] == comparison['llm_label']

# Overall accuracy summary
summary = pd.DataFrame({
    'Metric': ['Matches', 'Total', 'Accuracy'],
    'Value': [
        int(comparison['match'].sum()),
        len(comparison),
        f"{comparison['match'].mean():.2%}"
    ]
})

# Accuracy per label type (based on ground-truth rating)
per_type_accuracy = (
    comparison.groupby('rating', dropna=False)['match']
    .mean()
    .reset_index(name='accuracy')
    .sort_values('rating')
)
per_type_accuracy['accuracy'] = per_type_accuracy['accuracy'].map(lambda x: f"{x:.2%}")

summary, per_type_accuracy

(     Metric   Value
 0   Matches      36
 1     Total      80
 2  Accuracy  45.00%,
   rating accuracy
 0      A   95.24%
 1      L   35.71%
 2      R   24.44%)

In [13]:
%pip install tiktoken


Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/879.1 kB ? eta -:--:--
   ----------- ---------------------------- 262.1/879.1 kB ? eta -:--:--
   ---------------------------------------- 879.1/879.1 kB 10.2 MB/s  0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [15]:
prompt = f"""Classify the following tweet as LEFT, RIGHT, or APOLITICAL based on the ideological disposition of the speaker/writer.

DEFINITIONS:
- L: Any position from liberal democratic to communist, and anything in between. Includes liberal democrat, progressive, socialist, social democratic, anti-colonialist and anti-imperialist, anarchist, Marxist, internationalist, and communist ideologies.
- R: Any position from liberal conservative to fascist, and anything in between. Includes conservative, libertarian-right, authoritarian-right, monarchist, nationalist, anti-woke, religious conservatism, anti-globalism, racist, sexist, and fascist ideologies.
- A: Unopinionated, objective statements such as news reporting, can be false info with no opinion, scientific statements, factual observations, commonly held apolitical opinions or statements, or purely informational or advertising content with no ideological framing. However, tone and reason for pointing certain events out must be checked to see if the writer views this news as good or not potentially indicating a partisan leaning.

IMPORTANT GUIDANCE:
Judge based on the ideological disposition of the speaker/writer, not based on a checklist of specific positions. Take into account internet style subcultures, memes, coded language, and subcultural references that may convey ideological meaning beyond surface-level keywords. Consider tone, framing, and the broader context of online discourse. Answer the question based on who would write this?

Tweet to classify: 
Return ONLY ONE of the following words as your answer: L, R, A. Do not provide any additional explanation or text.
"""

In [16]:
import tiktoken

# Use the same model as your LLM calls
model_name = "gpt-4o-mini"

# Choose the text column used for classification
text_col = "summary" if "summary" in df.columns else "full_text"

# Pick tokenizer for the model (fallback to cl100k_base)
try:
    enc = tiktoken.encoding_for_model(model_name)
except KeyError:
    enc = tiktoken.get_encoding("cl100k_base")

# Ensure prompt string exists (defined in Cell 8)
if 'prompt' not in globals():
    raise ValueError("`prompt` is not defined. Run Cell 8 first.")

# Count prompt tokens once (same prompt used for every call)
prompt_tokens = len(enc.encode(prompt))

# Count input text tokens per row
input_token_counts = []
for text in df[text_col].fillna("").astype(str):
    input_token_counts.append(len(enc.encode(text)))

avg_input_tokens = sum(input_token_counts) / len(input_token_counts) if input_token_counts else 0

# Add prompt tokens + 1 output token per call
avg_tokens_per_call = avg_input_tokens + prompt_tokens + 1
estimated_total_tokens = int(
    sum(input_token_counts) + len(input_token_counts) * (prompt_tokens + 1)
)

print(f"Average tweet tokens ({text_col}): {avg_input_tokens:.2f}")
print(f"Prompt tokens: {prompt_tokens}")
print(f"Average tokens per call (tweet + prompt + 1 output token): {avg_tokens_per_call:.2f}")
print(f"Estimated total tokens for {len(input_token_counts)} calls: {estimated_total_tokens}")

Average tweet tokens (summary): 71.80
Prompt tokens: 322
Average tokens per call (tweet + prompt + 1 output token): 394.80
Estimated total tokens for 80 calls: 31584
